## Import Libraries

In [1]:
# Librerie per lettura file
import pandas as pd
from pathlib import Path
import json
import os
import psutil
import numpy as np
from tqdm import tqdm
import warnings
import joblib

#Kfold and GridSearch
from imblearn.over_sampling import SMOTE
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import GridSearchCV
from scipy.stats import randint, uniform


# Grafici
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib

# Data Analysis
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import (
    mean_absolute_error, mean_squared_error, r2_score,
    accuracy_score, precision_score, recall_score, f1_score,
    precision_recall_curve, classification_report, confusion_matrix,
    roc_curve, roc_auc_score, average_precision_score
)
from imblearn.over_sampling import SMOTE
from sklearn.multioutput import MultiOutputRegressor

# Modelli con Alberi
from xgboost import XGBRegressor, XGBClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier, GradientBoostingRegressor


from scipy.stats import uniform, randint

# Librerie Torch per MLP
import torch
import pytorch_lightning as pl
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split, SubsetRandomSampler
from torchmetrics import F1Score, Accuracy, Precision, Recall
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import CSVLogger

# Explainable AI (SHAP)
import shap

# Librerie per gestione dati parallela
import modin.pandas as mpd
import modin.config as cfg

# Visualizzazione matrice di confusione
from sklearn.metrics import ConfusionMatrixDisplay
import optuna
from sklearn.metrics import average_precision_score


In [2]:
matplotlib.use('Agg') 
warnings.filterwarnings("ignore", category=UserWarning, module="matplotlib")
optuna.logging.set_verbosity(optuna.logging.WARNING)
import warnings
warnings.filterwarnings(
    "ignore",
    message="FigureCanvasAgg is non-interactive, and thus cannot be shown"
)


## Reading File

It is necessary to have in the same directory the files obtained by ranking_best_drugs.ipynb so the cleaned transcriptomic and the ranked drugs.

In [3]:
# Reading the files
score_df = pd.read_csv("Preprocessing/drug_scores_with_targets.csv")
df_clean = pd.read_csv("Preprocessing/transcrittoma_pulito.csv")

In [4]:
score_df.iloc[185:200,:] #show the best 30 drugs (the first one has score None due to the lack of data, so it is not taken in consideration)

,Drug,Score,Drug_name,Targets
185,268.0,0.000936,YM155,['BIRC5']
186,166.0,0.000682,FTI-277,['FDPS']
187,194.0,0.000590,AUY922,['HSP90AA1']
188,1175.0,0.000514,AG-014699,"['PARP2', 'PARP1']"
189,135.0,0.000416,Gemcitabine,['DNA_damage']
190,88.0,0.000220,MS-275,"['O15379,P56524,Q13547,Q8WUI4,Q92769,Q96DB2,Q9..."
191,157.0,0.000211,JNK-9L,['JNK']
192,197.0,0.000141,Bryostatin 1,['PRKCA']
193,1527.0,0.000081,GDC0941 (rescreen),['PI3K']
194,87.0,-0.000111,GW843682X,['PLK1']


## Model Sensitivity

In this section, you can run models that predict drug sensitivity only. The analysis specifically focuses on the top 30 drugs, although you can include more if desired. However, note that drugs with a negative score perform worse than a random model and are therefore not recommended for use—this applies especially to drugs ranked beyond the 194th position.

Currently, only the functions used in the article are implemented here. However, models using scikit-learn can be easily added by modifying this structure, which is based on the XGBoost (XGB) model.

##### Choose the ensemble models 

models = [XGBClassifier for _ in range(5)]

##### Seed
rf_params = {
        "random_state": 42,
    }

##### Param grid space for Hyperparameter Tuning
params = {
    "n_estimators": randint(10, 301),              
    "max_depth": randint(3, 11),                     
    "learning_rate": [1,0.1,0.01,0.3],             
    "subsample": [0.4,0.5,0.6,0.8,1],                
    "colsample_bytree": [0.7,0.9,1],   
    "min_child_weight": randint(1, 11),          
    "gamma": [0,0.1,0.01,0.3,0.5]                         
}

##### Directory to save the results
dir = 'Results/Models_Sensitivity_prova/XGB'

#### Run the function
train_model(df_clean, rf_params,models, params, shap_dir = dir, l=31) #if you want more drugs to be evaluated increase l

### Functions for Sensitivity

#### Optuna objective

In [5]:
def objective(trial, model, X_train, y_train, X_val, y_val, base_params, param_grid):
    '''
    Objective function for hyperparameter optimization using Optuna.

    Parameters:
    - trial: Optuna Trial object, used to suggest new hyperparameter values.
    - model: scikit-learn-compatible model to be optimized.
    - X_train, y_train: training data.
    - X_val, y_val: validation data for performance evaluation.
    - base_params: dictionary of fixed model parameters (you can see it when you call the main function)
    - param_grid: dictionary defining the search space for each hyperparameter ((you can see it when you call the main function))

    Returns:
    - average_precision_score on the validation set, which will be maximized.
    '''

    params = base_params.copy()

    for key, space in param_grid.items():
        if isinstance(space, list):
            params[key] = trial.suggest_categorical(key, space)
        elif isinstance(space, range):
            params[key] = trial.suggest_int(key, min(space), max(space))
        elif hasattr(space, "rvs"):
            a, b = space.a, space.b
            params[key] = trial.suggest_int(key, a, b - 1)
        else:
            raise ValueError(f"Unsupported type for parameter {key}: {type(space)}")

    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict_proba(X_val)[:, 1]

    return average_precision_score(y_val, y_pred)

#### Model

In [6]:
def modello_base(train_set, train_cells, rf_params={}, 
               base_models=None, param_grid=None, n_splits=5, n_trials = 10, random_state=42):
    
    '''
    Function to train and evaluate ensemble ML models using K-Fold cross-validation as explained in the article,
    SMOTE for class balancing, and Optuna for hyperparameter optimization.

    Parameters:
    - train_set: DataFrame containing the full training data, including features and labels.
    - train_cells: Array-like list of cell line identifiers used for splitting the data into folds.
    - rf_params: Dictionary of fixed parameters for the base model(s) (optional).
    - base_models: List of model constructors (e.g., [RandomForestClassifier, ...]); one per fold. If None, RandomForestClassifier is used for all folds.
    - param_grid: Dictionary defining the search space for hyperparameters to tune with Optuna.
    - n_splits: Number of folds for cross-validation (default is 5).
    - n_trials: Number of Optuna trials per fold for hyperparameter tuning (default is 10).
    - random_state: Seed for reproducibility (used in SMOTE, CV splitting, and Optuna sampler).

    Returns:
    - models_per_fold: A list of dictionaries, each containing the trained model, evaluation scores (AUC, average precision, F1), and best hyperparameters for a fold.
    - y_val_all: Combined list of true validation labels across all folds.
    - y_val_pred_all: Combined list of predicted probabilities across all folds.
    '''

    #Base models if the models given to the function are None
    if base_models is None:
        base_models = [RandomForestClassifier for _ in range(n_splits)]
    
    #Parameter grid if the ones given to the function are None
    if param_grid is None:
        param_grid = {
            "n_estimators": [500],
            "max_depth": [7]
        }
    
    # Ensemble Models
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    models_per_fold = []
    y_val_all, y_val_pred_all = [], []
    i = 0

    for train_idx, val_idx in kf.split(train_cells):

        #Trainin and Validation for each split
        train_cells_fold = train_cells[train_idx]
        val_cells_fold = train_cells[val_idx]

        train_fold = train_set[train_set["Cell_line_cosmic_identifiers"].isin(train_cells_fold)]
        val_fold = train_set[train_set["Cell_line_cosmic_identifiers"].isin(val_cells_fold)]

        X_train = train_fold.drop(columns=["IC50", "Sensitivity", "Cell_line_cosmic_identifiers", "Screened_Compounds_", "Sample_Names", "Drug_id"])
        y_train = train_fold["Sensitivity"]

        X_val = val_fold.drop(columns=["IC50", "Sensitivity", "Cell_line_cosmic_identifiers", "Screened_Compounds_", "Sample_Names", "Drug_id"])
        y_val = val_fold["Sensitivity"]

        #Smote for balancing classes
        smote = SMOTE(random_state=42) 
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


        mod = base_models[i](**rf_params)

        sampler = optuna.samplers.TPESampler(seed=42)
        study = optuna.create_study(direction="maximize", sampler=sampler)

        study.optimize(
            lambda trial: objective(trial, mod, X_train_resampled, y_train_resampled, X_val, y_val, rf_params, param_grid),
            n_trials=n_trials
        )

        #Predict probability
        best_params = study.best_params
        best_params["random_state"] = 42
        rf_model = mod.set_params(**best_params)
        rf_model.fit(X_train_resampled, y_train_resampled)
        y_pred_prob = rf_model.predict_proba(X_val)[:, 1]


        y_val_all.extend(y_val)
        y_val_pred_all.extend(y_pred_prob)
        auc_score = roc_auc_score(y_val, y_pred_prob)
        auc_pr = average_precision_score(y_val, y_pred_prob)
        f1 = f1_score(y_val, (y_pred_prob > 0.4).astype(int))

        #Saving the metrics for the validation because auc_pr is used to predict the final probability as explained in the article
        models_per_fold.append({
            "model": rf_model,
            "auc_score": auc_score,
            "auc_pr": auc_pr,
            "f1_score": f1,
            "best_params": best_params
        })

        i += 1

    return models_per_fold, y_val_all, y_val_pred_all


#### Testing

In [7]:
def testing(train_set, train_cells, shap_dir, drug_name, n_splits=5, n_trials = 10, random_state=42):
    
    '''
    Function to test and evaluate already trained ensemble ML models using K-Fold cross-validation as explained in the article,
    SMOTE for class balancing, and Optuna for hyperparameter optimization.

    Parameters:
    - train_set: DataFrame containing the full training data, including features and labels.
    - train_cells: Array-like list of cell line identifiers used for splitting the data into folds.
    - shap_dir: directory where are saved the models
    – drug_name: name of the drug
    - n_splits: Number of folds for cross-validation (default is 5).
    - n_trials: Number of Optuna trials per fold for hyperparameter tuning (default is 10).
    - random_state: Seed for reproducibility (used in SMOTE, CV splitting, and Optuna sampler).

    Returns:
    - models_per_fold: A list of dictionaries, each containing the trained model, evaluation scores (AUC, average precision, F1), and best hyperparameters for a fold.
    - y_val_all: Combined list of true validation labels across all folds.
    - y_val_pred_all: Combined list of predicted probabilities across all folds.
    '''

    # Ensemble Models
    kf = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)
    models_per_fold = []
    y_val_all, y_val_pred_all = [], []
    i = 0

    for train_idx, val_idx in kf.split(train_cells):

        #Trainin and Validation for each split
        train_cells_fold = train_cells[train_idx]
        val_cells_fold = train_cells[val_idx]

        train_fold = train_set[train_set["Cell_line_cosmic_identifiers"].isin(train_cells_fold)]
        val_fold = train_set[train_set["Cell_line_cosmic_identifiers"].isin(val_cells_fold)]

        X_train = train_fold.drop(columns=["IC50", "Sensitivity", "Cell_line_cosmic_identifiers", "Screened_Compounds_", "Sample_Names", "Drug_id"])
        y_train = train_fold["Sensitivity"]

        X_val = val_fold.drop(columns=["IC50", "Sensitivity", "Cell_line_cosmic_identifiers", "Screened_Compounds_", "Sample_Names", "Drug_id"])
        y_val = val_fold["Sensitivity"]

        #Smote for balancing classes
        smote = SMOTE(random_state=42) 
        X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

        # Testing model already trained
        models_dir = os.path.join(shap_dir, f"{drug_name}_models")
        model_path = os.path.join(models_dir, f"model_fold_{i}.pkl")
        loaded_obj = joblib.load(model_path)
        model = loaded_obj['model']


        y_pred_prob = model.predict_proba(X_val)[:, 1]


        y_val_all.extend(y_val)
        y_val_pred_all.extend(y_pred_prob)
        auc_score = roc_auc_score(y_val, y_pred_prob)
        auc_pr = average_precision_score(y_val, y_pred_prob)
        f1 = f1_score(y_val, (y_pred_prob > 0.4).astype(int))

        #Saving the metrics for the validation because auc_pr is used to predict the final probability as explained in the article
        models_per_fold.append({
            "model":model,
            "auc_score": auc_score,
            "auc_pr": auc_pr,
            "f1_score": f1
        })

        i += 1

    return models_per_fold, y_val_all, y_val_pred_all


#### Plots

In [8]:
def plots(
    y_test,
    y_pred_test,
    y_val_all,
    y_val_pred_all,
    shap_dir,
    drug_name,
    specific_drug,
    y_pred_test_prob,
    all_fpr_test,
    all_precision_test,
    drug_names,
    recall_val,
    precision_val,
    avg_auc_score,
    auc_score_test,
    avg_auc_pr,
    auc_pr_test,
    best_threshold,
    cumulative_cm
):
        '''
        Function to generate plots
        '''
        cm = confusion_matrix(y_test, y_pred_test)
        cumulative_cm += cm
        disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["Negative", "Positive"])


        cm_fig_path = os.path.join(shap_dir, f"{drug_name}_confusion_matrix.png")
        disp.plot(cmap="Blues")
        plt.title(f"Confusion Matrix - {drug_name}")
        plt.tight_layout()
        plt.savefig(cm_fig_path)
        plt.close()

        fpr_val, tpr_val, _ = roc_curve(y_val_all, y_val_pred_all)
        fpr_test, tpr_test, _ = roc_curve(y_test, y_pred_test_prob)
        precision_test, recall_test, _ = precision_recall_curve(y_test, y_pred_test_prob)

        all_fpr_test.append((fpr_test, tpr_test))
        all_precision_test.append((recall_test, precision_test))
        drug_names.append(specific_drug)

        roc_fig_path = os.path.join(shap_dir, f"{drug_name}_roc_curve.png")
        plt.figure(figsize=(12, 5))

        plt.subplot(1, 2, 1)
        plt.plot(fpr_val, tpr_val, label=f'Validation AUC = {avg_auc_score:.3f}')
        plt.plot(fpr_test, tpr_test, label=f'Test AUC = {auc_score_test:.3f}')
        plt.plot([0, 1], [0, 1], 'k--')
        plt.xlabel("False Positive Rate")
        plt.ylabel("True Positive Rate")
        plt.title(f"ROC Curve - {drug_name}")
        plt.legend()

        plt.subplot(1, 2, 2)
        plt.plot(recall_val, precision_val, label=f'Validation AUC-PR = {avg_auc_pr:.3f}')
        plt.plot(recall_test, precision_test, label=f'Test AUC-PR = {auc_pr_test:.3f}')
        plt.axvline(x=best_threshold, color='r', linestyle="--", label=f"Best Threshold = {best_threshold:.3f}")
        plt.xlabel("Recall")
        plt.ylabel("Precision")
        plt.title(f"Precision-Recall Curve - {drug_name}")
        plt.legend()

        plt.tight_layout()
        plt.savefig(roc_fig_path)
        plt.close()

        cutoff_range = np.arange(y_pred_test_prob.min(), y_pred_test_prob.max(), 0.01)
        precisions_cutoff = []
        recalls_cutoff = []

        for cutoff in cutoff_range:
            predicted_sensitivity = (y_pred_test_prob > cutoff).astype(int)
            precision = precision_score(y_test, predicted_sensitivity, zero_division=0)
            recall = recall_score(y_test, predicted_sensitivity, zero_division=0)

            precisions_cutoff.append(precision)
            recalls_cutoff.append(recall)

        plt.figure(figsize=(7, 4))
        plt.plot(cutoff_range, precisions_cutoff, marker='o', label='Precision', color='blue')
        plt.plot(cutoff_range, recalls_cutoff, marker='x', label='Recall', color='orange')
        plt.title(f"Precision & Recall vs Probabilities Cut-off - {drug_name}")
        plt.xlabel("Probabilities Cut-off (on predicted values) ")
        plt.ylabel("Score")
        plt.ylim(0, 1.05)
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        plt.savefig(f"{shap_dir}/{drug_name}_precision_recall_curve_rf.png")
        plt.close()

#### SHAP

In [9]:
def generate_shap_plots(
    models_per_fold,
    X_test,
    test_set,
    df_clean,
    shap_dir,
    drug_name,
    specific_drug
):
        '''
        Function to compute SHAP and to generate plots
        '''

        # Directory SHAP
        shap_dir_drug = os.path.join(shap_dir, f"{drug_name}")
        os.makedirs(shap_dir_drug, exist_ok=True)  

        shap_values_list = []
        print(f"\nCalcolo SHAP per {drug_name} su {len(models_per_fold)} modelli...")


        ## Shap values for each model-cell_line
        for i, model_info in enumerate(models_per_fold):
            model = model_info["model"]
            explainer = shap.TreeExplainer(model)
            X_test_aligned = X_test[model.feature_names_in_]  
            shap_values = explainer.shap_values(X_test_aligned, check_additivity=False)
            if isinstance(shap_values, list):
                sv = shap_values[1] if len(shap_values) > 1 else shap_values[0]
            else:
                sv = shap_values

            if sv.shape[1] != X_test.shape[1]:
                print(f"⚠️ SHAP shape mismatch: got {sv.shape}, expected ({X_test.shape[0]}, {X_test.shape[1]})")
                continue

            shap_values_list.append(sv)

        ## MEAN SHAP for each model
        if shap_values_list:
            shap_df_list = []
            for sv in shap_values_list:
                if sv.ndim == 3:
                    sv = sv[:, :, 1]
                shap_df_list.append(pd.DataFrame(sv, columns=X_test.columns, index=X_test.index))

            mean_shap_df = sum(shap_df_list) / len(shap_df_list)

            for idx, row in X_test.iterrows():
       
                cell_line_id = test_set.loc[idx]["Cell_line_cosmic_identifiers"]
                matching_rows = df_clean[df_clean["Cell_line_cosmic_identifiers"] == cell_line_id]

                if not matching_rows.empty:
                    cell_name = matching_rows["cell_name"].iloc[0] 
                else:
                    print(f" Nessuna riga trovata per cell_line_id {cell_line_id}")
                    continue  

                shap_values_row = mean_shap_df.loc[idx]
                sorted_idx = np.argsort(np.abs(shap_values_row.values))[::-1]
                sorted_features = X_test.columns[sorted_idx]
                sorted_shap = shap_values_row.values[sorted_idx]
                sorted_vals = row.values[sorted_idx]

                if isinstance(explainer.expected_value, (list, np.ndarray)):
                        base_value = explainer.expected_value[1] if len(explainer.expected_value) > 1 else explainer.expected_value[0]
                else:
                        base_value = explainer.expected_value

                ## 1) Waterfall Plot
                shap.waterfall_plot(shap.Explanation(
                        values=sorted_shap,
                        base_values=base_value,  
                        data=sorted_vals,
                        feature_names=sorted_features
                    ))

                plt.suptitle(f"SHAP Waterfall - Cell Line: {cell_name}", fontsize=14)
                fig = plt.gcf()
                fig_path = os.path.join(shap_dir_drug, f"{cell_name}_shap_waterfall.png")
                fig.savefig(fig_path, format="png") 
                plt.close(fig)  


                ## 2) shap_bar_top 5 sensitivity and resistance
                shap_series = shap_values_row.sort_values()
                negativi = shap_series.head(5)
                positivi = shap_series.tail(5)
                combined = pd.concat([negativi, positivi])
                colori = ['lightcoral'] * 5 + ['skyblue'] * 5

                plt.figure(figsize=(5, 4))
                plt.barh(combined.index[::-1], combined.values[::-1], color=colori[::-1])
                plt.axvline(0, color='gray', linestyle='--')
                plt.title(f"Top 5 ± SHAP - {cell_name}", fontsize=10)
                plt.xlabel("SHAP Value (Effect)")
                plt.tight_layout()
                fig_bar_path = os.path.join(shap_dir_drug, f"{cell_name}_shap_bar_top5.png")
                plt.savefig(fig_bar_path, format="png")
                plt.close()

            ### Grafici SHAP sul farmaco ###

            ## shap_top5_mean_bar
            shap_mean = mean_shap_df.mean(axis=0).sort_values()
            top5_neg = shap_mean.head(5)
            top5_pos = shap_mean.tail(5)
            combined = pd.concat([top5_neg, top5_pos])
            colori = ['lightcoral'] * 5 + ['skyblue'] * 5
            plt.figure(figsize=(6, 4))
            plt.barh(combined.index[::-1], combined.values[::-1], color=colori[::-1])
            plt.axvline(0, color='gray', linestyle='--')
            plt.title(f"SHAP Top 5 ± - {drug_name} (media su tutte le cell)", fontsize=11)
            plt.xlabel("SHAP value (mean)")
            plt.tight_layout()
            fig_group_bar_path = os.path.join(shap_dir_drug, f"{drug_name}_shap_top5_mean_bar.png")
            plt.savefig(fig_group_bar_path, format="png")
            plt.close()

            abs_shap_df_list = [df.abs() for df in shap_df_list]
            mean_abs_shap_df = sum(abs_shap_df_list) / len(abs_shap_df_list)
            mean_abs_importance = mean_abs_shap_df.mean(axis=0)
            ## top 20 features
            top20_features = (
                mean_shap_df.abs().mean(axis=0)
                .sort_values(ascending=False)
                .head(20)
            )
            top20_features = mean_abs_importance.sort_values(ascending=False).head(20)
            top20_features_path = os.path.join(shap_dir_drug, "top20_features.csv")
            top20_features.to_csv(top20_features_path, header=["mean_abs_shap_value"])


            ## SHAP summary plot
            top10_indices = mean_abs_importance.sort_values(ascending=False).head(10).index
            top10_shap_values = mean_shap_df[top10_indices].values
            top10_X_test = X_test[top10_indices]

            plt.figure()
            shap.summary_plot(top10_shap_values, top10_X_test, feature_names=top10_indices, show=False)
            plt.title(f"SHAP Summary Plot (Top 10) - {drug_name}")
            plt.tight_layout()
            plt.savefig(os.path.join(shap_dir_drug, "shap_summary_plot.png"))
            plt.close()
            
            ## SHAP bar plot
            shap_bar_fig_path = os.path.join(shap_dir_drug, "shap_bar_plot.png")
            plt.figure()
            shap.summary_plot(mean_shap_df.values, X_test, feature_names=X_test.columns, plot_type="bar", show=False)
            plt.title(f"SHAP Bar Plot (Media) - {drug_name}")
            plt.tight_layout()
            plt.savefig(shap_bar_fig_path)
            plt.close()


#### Main function

In [10]:
def train_model(df_clean, rf_params, base_models, param_grid, shap_dir = 'RF_SENSITIVITY', l=31):
    '''
    Train models to predict drug sensitivity for multiple drugs,
    evaluate their performance, generate visualizations, and compute SHAP explanations.

    The function performs the following steps per drug:
    - Encodes categorical features.
    - Splits the dataset by cell lines into training and test sets.
    - Uses K-Fold cross-validation and SMOTE on training data.
    - Applies Optuna for hyperparameter optimization.
    - Evaluates on both validation and test sets (AUC, F1, Precision, Recall).
    - Saves trained models, performance metrics, confusion matrices, ROC/PR curves.
    - Computes SHAP values for test set samples and saves waterfall/summary plots.

    Parameters:
    - df_clean: Preprocessed and cleaned DataFrame containing features and labels.
    - rf_params: Dictionary of fixed parameters to use for RandomForestClassifier.
    - base_models: List of model classes to use per fold (e.g. [RandomForestClassifier, ...]).
    - param_grid: Dictionary of hyperparameters to tune via Optuna.
    - shap_dir: Directory where models, plots, and SHAP outputs will be saved (default 'RF_SENSITIVITY').
    - l: Number of drugs to process (default is 31; starts from index 1 of `score_df["Drug"]`).

    Returns:
    - Saves per-drug results and visualizations in the specified `shap_dir`.
    - Outputs a summary CSV (`rf_drug_sensitivity_results.csv`) with metrics per drug.
    - Prints final classification report and cumulative F1 score over all predictions.
    '''

    # Directory
    if not os.path.exists(shap_dir):
        os.makedirs(shap_dir)

    # Initialize variables
    label_encoders = {}
    df_encoded = df_clean.copy()
    df_encoded = df_encoded.dropna(subset=["Sensitivity"])
    for col in df_encoded.columns:
        if df_encoded[col].dtype == 'object' or df_encoded[col].dtype.name == 'category':
            le = LabelEncoder()
            df_encoded[col] = le.fit_transform(df_encoded[col].astype(str))
            label_encoders[col] = le

    models_results_rf_specific_drug_sensitivity = {}
    cumulative_cm = np.array([[0, 0], [0, 0]])
    all_fpr_test = []
    all_precision_test = []
    drug_names = []

    y_true_total = []
    y_pred_total = []

    for specific_drug in tqdm(score_df["Drug"].iloc[1:l], desc="Processing Drugs"):
        ## Model for each drug

        df_drug = df_encoded[df_encoded["Drug_id"] == specific_drug]
        drug_name = score_df.loc[score_df["Drug"] == specific_drug, "Drug_name"].values[0]

        # Directory for the specific drug
        models_dir = os.path.join(shap_dir, f"{drug_name}_models")
        os.makedirs(models_dir, exist_ok=True)

        # Training and Test set
        cell_counts = df_drug["Cell_line_cosmic_identifiers"].value_counts()
        train_cells, test_cells = train_test_split(
            cell_counts.index,
            test_size=0.2,
            stratify=df_drug.groupby("Cell_line_cosmic_identifiers")["Sensitivity"].apply(lambda x: x.mean()),
            random_state=42
        )

        train_set = df_drug[df_drug["Cell_line_cosmic_identifiers"].isin(train_cells)]
        test_set = df_drug[df_drug["Cell_line_cosmic_identifiers"].isin(test_cells)]

        X_test = test_set.drop(columns=["IC50", "Sensitivity", "Cell_line_cosmic_identifiers", "Screened_Compounds_", "Sample_Names", "Drug_id"])
        y_test = test_set["Sensitivity"]

        # Train the models and save them
        '''
        models_per_fold, y_val_all, y_val_pred_all = modello_base(train_set, train_cells, rf_params, 
               base_models, param_grid, n_splits=5, random_state=42)
        for fold_idx, model_info in enumerate(models_per_fold):
            model_path = os.path.join(models_dir, f"model_fold_{fold_idx}.pkl")
            joblib.dump(model_info, model_path)

        '''

        models_per_fold, y_val_all, y_val_pred_all = testing(train_set, train_cells, shap_dir, drug_name, n_splits=5, random_state=42)
        # Predict final probability
        avg_auc_score = np.mean([m["auc_score"] for m in models_per_fold])
        avg_auc_pr = np.mean([m["auc_pr"] for m in models_per_fold])
        avg_f1_score = np.mean([m["f1_score"] for m in models_per_fold])
        aucprs = np.array([m["auc_pr"] for m in models_per_fold])
        weights = aucprs / aucprs.sum()
        y_pred_test_prob = sum(weights[i] * models_per_fold[i]["model"].predict_proba(X_test)[:, 1] for i in range(len(weights)))

        #Calculate best threshol 
        precision_val, recall_val, thresholds = precision_recall_curve(y_val_all, y_val_pred_all)
        f1_scores = 2 * (precision_val * recall_val) / (precision_val + recall_val + 1e-6)
        best_threshold = thresholds[np.argmax(f1_scores)]
        y_pred_test = (y_pred_test_prob > best_threshold).astype(int)

        #Calculate metrics for the test
        auc_score_test = roc_auc_score(y_test, y_pred_test_prob)
        auc_pr_test = average_precision_score(y_test, y_pred_test_prob)
        f1_test = f1_score(y_test, y_pred_test)
        precision_test_val = precision_score(y_test, y_pred_test)
        recall_test_val = recall_score(y_test, y_pred_test)

        #Save all the metrics for the test
        models_results_rf_specific_drug_sensitivity[drug_name] = {
            "avg_auc_score": avg_auc_score,
            "avg_auc_pr": avg_auc_pr,
            "avg_f1_score": avg_f1_score,
            "auc_score_test": auc_score_test,
            "auc_pr_test": auc_pr_test,
            "f1_score_test": f1_test,
            "best_threshold": best_threshold,
            "precision_test": precision_test_val,
            "recall_test": recall_test_val
        }

        ## Generate plots
        plots(y_test,
            y_pred_test,
            y_val_all,
            y_val_pred_all,
            shap_dir,
            drug_name,
            specific_drug,
            y_pred_test_prob,
            all_fpr_test,
            all_precision_test,
            drug_names,
            recall_val,
            precision_val,
            avg_auc_score,
            auc_score_test,
            avg_auc_pr,
            auc_pr_test,
            best_threshold,
            cumulative_cm
        )

        # SHAP and plots
        generate_shap_plots(
            models_per_fold,
            X_test,
            test_set,
            df_clean,
            shap_dir,
            drug_name,
            specific_drug
        )

    
        # Print results (Not necessary)
        print(f"\n **Results for {drug_name}** ")
        print(f"AUC-ROC Test: {auc_score_test:.3f}")
        print(f"AUC-PR Test: {auc_pr_test:.3f}")
        print(f"F1-score Test: {f1_test:.3f}")

        print("\n **Validation Set Classification Report:**")
        print(classification_report(y_val_all, (np.array(y_val_pred_all) > 0.5).astype(int)))

        print("\n**Test Set Classification Report:**")
        print(classification_report(y_test, y_pred_test))

        y_true_total.extend(y_test)
        y_pred_total.extend(y_pred_test)


    # Print and Save Final results
    print("\n\n========== RIASSUNTO FINALE ==========")
    
    final_f1 = f1_score(y_true_total, y_pred_total)
    results_df = pd.DataFrame.from_dict(models_results_rf_specific_drug_sensitivity, orient="index")
    results_df.to_csv(os.path.join(shap_dir, "rf_drug_sensitivity_results.csv"))

    cm_fig_path = os.path.join(shap_dir, "total_confusion_matrix.png")
    disp = ConfusionMatrixDisplay(confusion_matrix=cumulative_cm, display_labels=["Negative", "Positive"])
    fig, ax = plt.subplots()
    disp.plot(cmap="Purples", ax=ax)
    plt.title(f"Confusion Matrix - TOTALE (Test Set)\nF1-score Finale: {final_f1:.4f}")
    plt.tight_layout()
    plt.savefig(cm_fig_path)
    plt.close()

    roc_all_fig_path = os.path.join(shap_dir, "roc_curve_all_drugs.png")
    plt.figure(figsize=(10, 5))
    for i, (fpr, tpr) in enumerate(all_fpr_test):
        plt.plot(fpr, tpr, label=drug_names[i])
    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve - Tutti i farmaci")
    plt.legend()
    plt.tight_layout()
    plt.savefig(roc_all_fig_path)
    plt.close()

    pr_all_fig_path = os.path.join(shap_dir, "precision_recall_curve_all_drugs.png")
    plt.figure(figsize=(10, 5))
    for i, (recall, precision) in enumerate(all_precision_test):
        plt.plot(recall, precision, label=drug_names[i])
    plt.xlabel("Recall")
    plt.ylabel("Precision")
    plt.title("Precision-Recall Curve - Tutti i farmaci")
    plt.legend()
    plt.tight_layout()
    plt.savefig(pr_all_fig_path)
    plt.close()

#### Model Random Forest

In [24]:
# Seed
rf_params = {
        "random_state": 1,
    }

# Param grid space for Hyperparameter Tuning (it can be changed)
param_grid = {
    "n_estimators": randint(10, 501),              
    "max_depth": [5, 7, 10, 20, None],
    "min_samples_split": randint(2, 11),
    "min_samples_leaf": randint(1, 6),
    "max_features": ["sqrt", "log2"]
}

# Directory to save the results
dir = 'Results/Models_Sensitivity/RF'

# Run the function
train_model(df_clean, rf_params, None, param_grid, dir, l=31) #if you want more drugs to be evaluated, increase l.

Processing Drugs:   0%|          | 0/30 [00:00<?, ?it/s]


Calcolo SHAP per KIN001-260 su 5 modelli...


Processing Drugs:   3%|▎         | 1/30 [00:35<17:23, 35.97s/it]


 **Results for KIN001-260** 
AUC-ROC Test: 0.754
AUC-PR Test: 0.597
F1-score Test: 0.512

 **Validation Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.87      0.93      0.90       351
         1.0       0.73      0.60      0.66       119

    accuracy                           0.84       470
   macro avg       0.80      0.76      0.78       470
weighted avg       0.84      0.84      0.84       470


**Test Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.85      0.94      0.89        93
         1.0       0.65      0.42      0.51        26

    accuracy                           0.82       119
   macro avg       0.75      0.68      0.70       119
weighted avg       0.81      0.82      0.81       119


Calcolo SHAP per TG101348 su 5 modelli...


Processing Drugs:   7%|▋         | 2/30 [01:12<16:48, 36.04s/it]

...


 **Results for PIK-93** 
AUC-ROC Test: 0.843
AUC-PR Test: 0.754
F1-score Test: 0.667

 **Validation Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.94      0.93      0.93       804
         1.0       0.60      0.65      0.62       136

    accuracy                           0.89       940
   macro avg       0.77      0.79      0.78       940
weighted avg       0.89      0.89      0.89       940


**Test Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.96      0.95      0.96       212
         1.0       0.64      0.69      0.67        26

    accuracy                           0.92       238
   macro avg       0.80      0.82      0.81       238
weighted avg       0.93      0.92      0.93       238


Calcolo SHAP per XMD15-27 su 5 modelli...


Processing Drugs:  97%|█████████▋| 29/30 [20:38<01:11, 71.90s/it]


 **Results for XMD15-27** 
AUC-ROC Test: 0.956
AUC-PR Test: 0.856
F1-score Test: 0.757

 **Validation Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.94      0.94      0.94      2045
         1.0       0.58      0.57      0.57       300

    accuracy                           0.89      2345
   macro avg       0.76      0.75      0.75      2345
weighted avg       0.89      0.89      0.89      2345


**Test Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.98      0.93      0.96       515
         1.0       0.67      0.88      0.76        80

    accuracy                           0.92       595
   macro avg       0.82      0.90      0.86       595
weighted avg       0.94      0.92      0.93       595


Calcolo SHAP per AC220 su 5 modelli...


Processing Drugs: 100%|██████████| 30/30 [21:07<00:00, 42.26s/it]


 **Results for AC220** 
AUC-ROC Test: 0.832
AUC-PR Test: 0.469
F1-score Test: 0.357

 **Validation Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.91      0.89      0.90       397
         1.0       0.47      0.54      0.50        72

    accuracy                           0.84       469
   macro avg       0.69      0.72      0.70       469
weighted avg       0.85      0.84      0.84       469


**Test Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.91      0.92      0.91       104
         1.0       0.38      0.33      0.36        15

    accuracy                           0.85       119
   macro avg       0.65      0.63      0.64       119
weighted avg       0.84      0.85      0.84       119



========== RIASSUNTO FINALE ==========


#### XGBoosting

In [ ]:
#Choose the ensemble models 
models = [XGBClassifier for _ in range(5)]

# Seed
rf_params = {
        "random_state": 42,
    }

# Param grid space for Hyperparameter Tuning (it can be changed)
params = {
    "n_estimators": randint(10, 301),              
    "max_depth": randint(3, 11),                     
    "learning_rate": [1,0.1,0.01,0.3],             
    "subsample": [0.4,0.5,0.6,0.8,1],                
    "colsample_bytree": [0.7,0.9,1],   
    "min_child_weight": randint(1, 11),          
    "gamma": [0,0.1,0.01,0.3,0.5]                         
}

# Directory to save the results
dir = 'Results/Models_Sensitivity/XGB'

#Run the function
train_model(df_clean, rf_params,models, params, shap_dir = dir, l=31)

Processing Drugs:   0%|          | 0/30 [00:00<?, ?it/s]


Calcolo SHAP per KIN001-260 su 5 modelli...


Processing Drugs:   3%|▎         | 1/30 [00:25<12:27, 25.76s/it]


 **Results for KIN001-260** 
AUC-ROC Test: 0.734
AUC-PR Test: 0.597
F1-score Test: 0.489

 **Validation Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.87      0.90      0.88       351
         1.0       0.67      0.60      0.63       119

    accuracy                           0.82       470
   macro avg       0.77      0.75      0.76       470
weighted avg       0.82      0.82      0.82       470


**Test Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.85      0.91      0.88        93
         1.0       0.58      0.42      0.49        26

    accuracy                           0.81       119
   macro avg       0.71      0.67      0.68       119
weighted avg       0.79      0.81      0.80       119


Calcolo SHAP per TG101348 su 5 modelli...


Processing Drugs:   7%|▋         | 2/30 [00:50<11:48, 25.29s/it]

...


 **Results for PIK-93** 
AUC-ROC Test: 0.806
AUC-PR Test: 0.597
F1-score Test: 0.667

 **Validation Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.94      0.92      0.93       804
         1.0       0.58      0.67      0.62       136

    accuracy                           0.88       940
   macro avg       0.76      0.79      0.77       940
weighted avg       0.89      0.88      0.88       940


**Test Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.96      0.95      0.96       212
         1.0       0.64      0.69      0.67        26

    accuracy                           0.92       238
   macro avg       0.80      0.82      0.81       238
weighted avg       0.93      0.92      0.93       238


Calcolo SHAP per XMD15-27 su 5 modelli...


Processing Drugs:  97%|█████████▋| 29/30 [16:56<01:01, 61.84s/it]


 **Results for XMD15-27** 
AUC-ROC Test: 0.954
AUC-PR Test: 0.857
F1-score Test: 0.759

 **Validation Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.93      0.90      0.92      2045
         1.0       0.44      0.52      0.48       300

    accuracy                           0.86      2345
   macro avg       0.69      0.71      0.70      2345
weighted avg       0.87      0.86      0.86      2345


**Test Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.95      0.98      0.97       515
         1.0       0.85      0.69      0.76        80

    accuracy                           0.94       595
   macro avg       0.90      0.83      0.86       595
weighted avg       0.94      0.94      0.94       595


Calcolo SHAP per AC220 su 5 modelli...


Processing Drugs: 100%|██████████| 30/30 [17:24<00:00, 34.80s/it]


 **Results for AC220** 
AUC-ROC Test: 0.781
AUC-PR Test: 0.360
F1-score Test: 0.412

 **Validation Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.92      0.87      0.90       397
         1.0       0.46      0.58      0.51        72

    accuracy                           0.83       469
   macro avg       0.69      0.73      0.70       469
weighted avg       0.85      0.83      0.84       469


**Test Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.92      0.88      0.90       104
         1.0       0.37      0.47      0.41        15

    accuracy                           0.83       119
   macro avg       0.64      0.68      0.66       119
weighted avg       0.85      0.83      0.84       119



========== RIASSUNTO FINALE ==========


#### Gradient Boosting

In [ ]:
models = [GradientBoostingClassifier for _ in range(5)]

#Seed
rf_params = {
        "random_state": 1,
    }

# Hyp Tuning
params = {
    "n_estimators": randint(10, 301),            
    "max_depth": randint(3, 11),                   
    "learning_rate": [1, 0.1, 0.01, 0.3],          
    "subsample": [0.4, 0.5, 0.6, 0.8, 1],          
    "min_samples_split": randint(2, 11),          
    "min_samples_leaf": randint(1, 11),            
    "max_features": ['sqrt', 'log2', None]         
}

# Directory to save the results
dir = 'Results/Models_Sensitivity/Gradient_Boosting'

#Run the function
train_model(df_clean, rf_params, models, params, shap_dir = dir, l=31)

Processing Drugs:   0%|          | 0/30 [00:00<?, ?it/s]


Calcolo SHAP per KIN001-260 su 5 modelli...


Processing Drugs:   3%|▎         | 1/30 [00:25<12:26, 25.74s/it]


 **Results for KIN001-260** 
AUC-ROC Test: 0.722
AUC-PR Test: 0.553
F1-score Test: 0.512

 **Validation Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.87      0.93      0.90       351
         1.0       0.75      0.58      0.65       119

    accuracy                           0.84       470
   macro avg       0.81      0.76      0.78       470
weighted avg       0.84      0.84      0.84       470


**Test Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.85      0.94      0.89        93
         1.0       0.65      0.42      0.51        26

    accuracy                           0.82       119
   macro avg       0.75      0.68      0.70       119
weighted avg       0.81      0.82      0.81       119


Calcolo SHAP per TG101348 su 5 modelli...


Processing Drugs:   7%|▋         | 2/30 [00:51<11:59, 25.69s/it]

...


 **Results for PIK-93** 
AUC-ROC Test: 0.816
AUC-PR Test: 0.679
F1-score Test: 0.640

 **Validation Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.93      0.95      0.94       804
         1.0       0.65      0.57      0.61       136

    accuracy                           0.89       940
   macro avg       0.79      0.76      0.77       940
weighted avg       0.89      0.89      0.89       940


**Test Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.95      0.96      0.96       212
         1.0       0.67      0.62      0.64        26

    accuracy                           0.92       238
   macro avg       0.81      0.79      0.80       238
weighted avg       0.92      0.92      0.92       238


Calcolo SHAP per XMD15-27 su 5 modelli...


Processing Drugs:  97%|█████████▋| 29/30 [20:46<01:24, 84.46s/it]


 **Results for XMD15-27** 
AUC-ROC Test: 0.965
AUC-PR Test: 0.821
F1-score Test: 0.778

 **Validation Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.94      0.94      0.94      2045
         1.0       0.61      0.62      0.61       300

    accuracy                           0.90      2345
   macro avg       0.78      0.78      0.78      2345
weighted avg       0.90      0.90      0.90      2345


**Test Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.98      0.94      0.96       515
         1.0       0.70      0.88      0.78        80

    accuracy                           0.93       595
   macro avg       0.84      0.91      0.87       595
weighted avg       0.94      0.93      0.94       595


Calcolo SHAP per AC220 su 5 modelli...


Processing Drugs: 100%|██████████| 30/30 [21:21<00:00, 42.72s/it]


 **Results for AC220** 
AUC-ROC Test: 0.799
AUC-PR Test: 0.455
F1-score Test: 0.452

 **Validation Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.90      0.92      0.91       397
         1.0       0.52      0.44      0.48        72

    accuracy                           0.85       469
   macro avg       0.71      0.68      0.70       469
weighted avg       0.84      0.85      0.85       469


**Test Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.92      0.91      0.92       104
         1.0       0.44      0.47      0.45        15

    accuracy                           0.86       119
   macro avg       0.68      0.69      0.68       119
weighted avg       0.86      0.86      0.86       119



========== RIASSUNTO FINALE ==========


#### GB_XGB_RF

In [11]:
models = [XGBClassifier, GradientBoostingClassifier, RandomForestClassifier, RandomForestClassifier,XGBClassifier ]

#Seed
rf_params = {
        "random_state": 42,
    }

#Hyp Tuning
param_grid = {
    "n_estimators": randint(10, 501),              
    "max_depth": [5, 7, 10, 20, None]
}

#Directory
dir = 'Results/Models_Sensitivity/XGB_RF_GB'

#Run
train_model(df_clean, rf_params, models, param_grid, dir, l=31)

Processing Drugs:   0%|          | 0/30 [00:00<?, ?it/s]


Calcolo SHAP per KIN001-260 su 5 modelli...


Processing Drugs:   3%|▎         | 1/30 [00:40<19:29, 40.32s/it]


 **Results for KIN001-260** 
AUC-ROC Test: 0.748
AUC-PR Test: 0.598
F1-score Test: 0.565

 **Validation Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.88      0.94      0.91       351
         1.0       0.78      0.61      0.69       119

    accuracy                           0.86       470
   macro avg       0.83      0.78      0.80       470
weighted avg       0.85      0.86      0.85       470


**Test Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.87      0.92      0.90        93
         1.0       0.65      0.50      0.57        26

    accuracy                           0.83       119
   macro avg       0.76      0.71      0.73       119
weighted avg       0.82      0.83      0.82       119


Calcolo SHAP per TG101348 su 5 modelli...


Processing Drugs:   7%|▋         | 2/30 [01:27<20:49, 44.63s/it]

...


 **Results for PIK-93** 
AUC-ROC Test: 0.862
AUC-PR Test: 0.769
F1-score Test: 0.667

 **Validation Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.93      0.95      0.94       804
         1.0       0.66      0.56      0.60       136

    accuracy                           0.89       940
   macro avg       0.79      0.75      0.77       940
weighted avg       0.89      0.89      0.89       940


**Test Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.96      0.95      0.96       212
         1.0       0.64      0.69      0.67        26

    accuracy                           0.92       238
   macro avg       0.80      0.82      0.81       238
weighted avg       0.93      0.92      0.93       238


Calcolo SHAP per XMD15-27 su 5 modelli...


Processing Drugs:  97%|█████████▋| 29/30 [21:31<01:12, 72.03s/it]


 **Results for XMD15-27** 
AUC-ROC Test: 0.944
AUC-PR Test: 0.813
F1-score Test: 0.688

 **Validation Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.94      0.93      0.93      2045
         1.0       0.56      0.58      0.57       300

    accuracy                           0.89      2345
   macro avg       0.75      0.76      0.75      2345
weighted avg       0.89      0.89      0.89      2345


**Test Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.95      0.95      0.95       515
         1.0       0.69      0.69      0.69        80

    accuracy                           0.92       595
   macro avg       0.82      0.82      0.82       595
weighted avg       0.92      0.92      0.92       595


Calcolo SHAP per AC220 su 5 modelli...


Processing Drugs: 100%|██████████| 30/30 [22:01<00:00, 44.06s/it]


 **Results for AC220** 
AUC-ROC Test: 0.809
AUC-PR Test: 0.453
F1-score Test: 0.400

 **Validation Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.92      0.91      0.92       397
         1.0       0.53      0.56      0.54        72

    accuracy                           0.86       469
   macro avg       0.73      0.73      0.73       469
weighted avg       0.86      0.86      0.86       469


**Test Set Classification Report:**
              precision    recall  f1-score   support

         0.0       0.92      0.88      0.90       104
         1.0       0.35      0.47      0.40        15

    accuracy                           0.82       119
   macro avg       0.63      0.67      0.65       119
weighted avg       0.85      0.82      0.83       119



========== RIASSUNTO FINALE ==========
